In [10]:
import numpy as np
import os
import pickle
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.callbacks import ModelCheckpoint
from string import punctuation
import datetime
import tensorflow as tf
from tqdm import tqdm

In [17]:
f = open('songs.dat', encoding="utf-8")

In [18]:
text = f.read()

In [30]:
text = text.lower().replace("\n\n", "\n")

In [32]:
text = text.translate(str.maketrans("", "", punctuation))

In [4]:
print(text[:5000])


טוב לי כשאת נמצאת פה לידי
ואת רוקדת לצידי
איזו ילדה איזו יפה
אנו רוקדים כל כך צמודים
טוב לי כשאת פה אהובתי
ואת אוחזת בידי
איזו ילדה איזו יפה
אנו רוקדים כל כך צמודים
לא לא רוצה שזה יגמר
שלעולם תהיה איתי
ושלנצח כך נשאר
ושנהיה תמיד אוהבים
בואי ילדונת בואי יפה
שנינו ביחד סיפור אהבה
בואי אלי בואי ילדה
את לי האושר האור בעיני
פעם הייתי בודד כל כך
עד שהגעת והכל נשכח
כמו חלום מתוק של אושר וצחוק
רק למראך ליבי נפתח
לא לא רוצה שזה יגמר
שלעולם תהיה איתי
ושלנצח כך נשאר
ושנהיה תמיד אוהבים
טוב לי כשאת נמצאת פה לידי
ואת רוקדת לצידי
איזו ילדה איזו יפה
אנו רוקדים כל כך צמודים
טוב לי כשאת פה אהובתי
ואת אוחזת בידי
איזו ילדה איזו יפה
אנו רוקדים כל כך צמודים
לא לא רוצה שזה יגמר
שלעולם תהיה איתי
ושלנצח כך נשאר
ושנהיה תמיד אוהבים
בואי ילדונת בואי יפה
שנינו ביחד סיפור אהבה
בואי אלי בואי ילדה
את לי האושר האור בעיני
פעם הייתי בודד כל כך
עד שהגעת והכל נשכח
כמו חלום מתוק של אושר וצחוק
רק למראך ליבי נפתח
לא לא רוצה שזה יגמר
שלעולם תהיה איתי
ושלנצח כך נשאר
ושנהיה תמיד אוהבים
לא נפקד מקומך מהלילה 
מעולם לא נפקד מקומך

In [34]:
with open('new_text1.txt', 'w', encoding='utf-8') as new_file:
    new_file.write(text)

In [2]:
with open('new_text1.txt', 'r', encoding='utf-8') as new_file:
    text = new_file.read()

In [5]:
text = text[:100000]
n_chars = len(text)
unique_chars = ''.join(sorted(set(text)))
print("unique_chars:", unique_chars)
n_unique_chars = len(unique_chars)
print("Number of characters:", n_chars)
print("Number of unique characters:", n_unique_chars)

unique_chars: 
 abcdefgilmnopqrstuvyְֲִֶַָּאבגדהוזחטיךכלםמןנסעףפץצקרשת‟
Number of characters: 100000
Number of unique characters: 57


In [6]:
# dictionary that converts characters to integers
char2int = {c: i for i, c in enumerate(unique_chars)}
# dictionary that converts integers to characters
int2char = {i: c for i, c in enumerate(unique_chars)}

In [7]:
# save these dictionaries for later generation
pickle.dump(char2int, open("char2int.pickle", "wb"))
pickle.dump(int2char, open("int2char.pickle", "wb"))

In [8]:
# hyper parameters
sequence_length = 100
step = 1
batch_size = 256
epochs = 40
sentences = []
y_train = []
for i in range(0, len(text) - sequence_length, step):
    sentences.append(text[i: i + sequence_length])
    y_train.append(text[i+sequence_length])
print("Number of sentences:", len(sentences))

Number of sentences: 99900


In [83]:
sentences[1]

'טוב לי כשאת נמצאת פה לידי\nואת רוקדת לצידי\nאיזו ילדה איזו יפה\nאנו רוקדים כל כך צמודים\nטוב לי כשאת פה '

In [11]:
# vectorization
X = np.zeros((len(sentences), sequence_length, n_unique_chars))
y = np.zeros((len(sentences), n_unique_chars))

for i, sentence in tqdm(enumerate(sentences)):
    for t, char in enumerate(sentence):
        X[i, t, char2int[char]] = 1
        y[i, char2int[y_train[i]]] = 1
print("X.shape:", X.shape)
print("y.shape:", y.shape)

99900it [00:09, 10926.39it/s]

X.shape: (99900, 100, 57)
y.shape: (99900, 57)


In [87]:
X[0][0]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])

In [12]:
# building the model
model = Sequential([
    LSTM(128, input_shape=(sequence_length, n_unique_chars)),
    Dense(n_unique_chars, activation="softmax"),
])

In [41]:
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [13]:
model.summary()
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
# make results folder if does not exist yet
if not os.path.isdir("results"):
    os.mkdir("results")
# save the model in each epoch
checkpoint = ModelCheckpoint("results/mizrahit-{loss:.2f}.h5", verbose=1)
model.fit(X, y, batch_size=batch_size, epochs=epochs, callbacks=[checkpoint, ])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               95232     
_________________________________________________________________
dense_1 (Dense)              (None, 57)                7353      
Total params: 102,585
Trainable params: 102,585
Non-trainable params: 0
_________________________________________________________________


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/40





99900/99900 [==============================] - 206s 2ms/step - loss: 2.6253 - acc: 0.2813

Epoch 00001: saving model to results/mizrahit-2.63.h5
Epoch 2/40
99900/99900 [==============================] - 216s 2ms/step - loss: 2.3148 - acc: 0.3435

Epoch 00002: saving model to results/mizrahit-2.31.h5
Epoch 3/40
99900/99900 [==============================] - 283s 3ms/step - loss: 2.1977 - acc: 0.3687

Epoch 00003: s

99900/99900 [==============================] - 288s 3ms/step - loss: 1.3285 - acc: 0.6161

Epoch 00034: saving model to results/mizrahit-1.33.h5
Epoch 35/40
99900/99900 [==============================] - 568s 6ms/step - loss: 1.3149 - acc: 0.6202

Epoch 00035: saving model to results/mizrahit-1.31.h5
Epoch 36/40
99900/99900 [==============================] - 373s 4ms/step - loss: 1.3020 - acc: 0.6238

Epoch 00036: saving model to results/mizrahit-1.30.h5
Epoch 37/40
99900/99900 [==============================] - 324s 3ms/step - loss: 1.2905 - acc: 0.6270

Epoch 00037: saving model to results/mizrahit-1.29.h5
Epoch 38/40
99900/99900 [==============================] - 310s 3ms/step - loss: 1.2792 - acc: 0.6312

Epoch 00038: saving model to results/mizrahit-1.28.h5
Epoch 39/40
99900/99900 [==============================] - 287s 3ms/step - loss: 1.2665 - acc: 0.6345

Epoch 00039: saving model to results/mizrahit-1.27.h5
Epoch 40/40
99900/99900 [==============================] - 280s 3ms/st